In [41]:
# MINI TEST
import pandas as pd
import os
import numpy as np
import shapely as shp
import geopandas as gpd

import pulp
import scipy as sp
from geopy.distance import great_circle
from geopy.distance import geodesic

# import plotly.plotly as py
# import plotly.graph_objs as go
# from matplotlib import rc

os.chdir("/Users/anayahall/projects/grapevine")

In [108]:
# mini gdfs of county wastes (tbm - location and MSW for 2014) 
c = gpd.read_file("data/clean/techbiomass_pts.shp")
#c = c[(c['biomass.ca'] == "organic fraction municipal solid waste") & (c['year'] == 2014)].copy()
c = c[(c['biomass.fe'] == "FOOD") & (c['year'] == 2014)].copy()

c = c[['FIPS', 'COUNTY', 'disposal.y', 'geometry']]
# subset out four counties
csub = c[(c['COUNTY'] == "Los Angeles") | (c['COUNTY'] == "San Diego")| 
         (c['COUNTY'] == "Orange")| (c['COUNTY'] == "Imperial")].copy()
# csub = c[(c['COUNTY'] == "San Diego")| (c['COUNTY'] == "Imperial")].copy()

csub


,FIPS,COUNTY,disposal.y,geometry
56,6037,Los Angeles,307295.084576,POINT (-118.5720592538903 34.14803912013723)
3206,6059,Orange,104525.755860,POINT (-117.8563081974633 33.53332689323803)
4106,6025,Imperial,6751.167110,POINT (-115.5613145428947 32.79849442795177)
5906,6073,San Diego,113139.598593,POINT (-117.2241057245406 32.94260371968307)


In [109]:
####MAKE DICTIONARY HERE
cdict = dict(zip(csub['COUNTY'], csub['disposal.y']))
cdict

{'Imperial': 6751.16711032454,
 'Los Angeles': 307295.084575734,
 'Orange': 104525.755860299,
 'San Diego': 113139.59859294}

In [120]:
# Mini gdfs of facilites (location and capacity)
f = gpd.read_file("data/clean/clean_swis.shp")
# f.head(8)
f = f[['SwisNo', 'AcceptedWa', 'County', 'cap_m3', 'geometry']].copy()

# subset out four counties
fsub = f[(f['County'] == "Los Angeles") | (f['County'] == "San Diego") | 
          (f['County'] == "Orange")| (f['County'] == "Imperial")].copy()
# too many, just select first 5
fsub = fsub[0:5].copy()
fdict = dict(zip(fsub['SwisNo'], fsub['cap_m3'])) #alt is use SwisNo

facnames = []
for key, value in fdict.items():
#     print(key)
    facnames.append(key)
facnames


['30-AB-0378', '19-AR-5584', '37-AB-0011', '13-AA-0095', '19-AA-1060']

In [123]:
fdict
# come back to distance matrix - use fake data for now

{'13-AA-0095': 54803.3024,
 '19-AA-1060': 9174.66,
 '19-AR-5584': 33823.9132,
 '30-AB-0378': 7645.55,
 '37-AB-0011': 7645.55}

In [49]:
# try using LP solver...................

In [124]:
#FIRST RUN TEST - BASED ON BEER DISTRIBUTION EXAMPLE
# Import PuLP modeler functions
from pulp import *

# Creates a list of all the supply nodes
Counties  = ["Imperial", "Los Angeles", "Orange", "San Diego"]

# Creates a dictionary for the number of units of supply for each supply node
waste = cdict

# Creates a list of all demand nodes
Facilities = facnames

# Creates a dictionary for the number of units of demand for each demand node
compost = fdict

# Creates a list of costs of each transportation path
costs = [   #Fac
         #2 3 6 10 11
         [2,4,5,2,1],#Imp   Counties
         [3,1,3,2,3], #LA
         [6,1,8,2,5], #ORANGE
         [3,4,1,5,3] #SD
        ]

# The cost data is made into a dictionary
costs = makeDict([Counties, Facilities],costs,0)

emfac = 1.8

# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Compost Distribution Problem",LpMaximize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(c,f) for c in Counties for f in Facilities]

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route",(Counties,Facilities),0,None,LpInteger)

# The objective function is added to 'prob' first
prob += lpSum([vars[c][f]*costs[c][f]*emfac for (c,f) in Routes]), "Sum_of_Transporting_Costs"

# # The supply maximum constraints are added to prob for each supply node (warehouse)
for c in Counties:
    prob += lpSum([vars[c][f] for f in Facilities])<=waste[c], "Sum_of_waste_out_of_Counties_%s"%c

# The demand minimum constraints are added to prob for each demand node (bar)
for f in Facilities:
    prob += lpSum([vars[c][f] for c in Counties])<=compost[f], "Sum_of_compost_into_Facilities_%s"%f

# vars 
# The problem data is written to an .lp file
prob.writeLP("CompostDistributionProblem.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(prob.objective))


Status: Optimal
Route_Imperial_13_AA_0095 = 0.0
Route_Imperial_19_AA_1060 = 0.0
Route_Imperial_19_AR_5584 = 6750.0
Route_Imperial_30_AB_0378 = 0.0
Route_Imperial_37_AB_0011 = 0.0
Route_Los_Angeles_13_AA_0095 = 0.0
Route_Los_Angeles_19_AA_1060 = 0.0
Route_Los_Angeles_19_AR_5584 = 0.0
Route_Los_Angeles_30_AB_0378 = 0.0
Route_Los_Angeles_37_AB_0011 = 0.0
Route_Orange_13_AA_0095 = 0.0
Route_Orange_19_AA_1060 = 9174.0
Route_Orange_19_AR_5584 = 0.0
Route_Orange_30_AB_0378 = 7645.0
Route_Orange_37_AB_0011 = 7645.0
Route_San_Diego_13_AA_0095 = 54803.0
Route_San_Diego_19_AA_1060 = 0.0
Route_San_Diego_19_AR_5584 = 27073.0
Route_San_Diego_30_AB_0378 = 0.0
Route_San_Diego_37_AB_0011 = 0.0
Total Cost of Transportation =  1011972.6


In [19]:
"""
The American Steel Problem for the PuLP Modeller
Authors: Antony Phillips, Dr Stuart Mitchell  2007
"""

# Import PuLP modeller functions
from pulp import *

# List of all the nodes
Nodes = ["Youngstown",
         "Pittsburgh",
         "Cincinatti",
         "Kansas City",
         "Chicago",
         "Albany",
         "Houston",
         "Tempe",
         "Gary"]

nodeData = {# NODE        Supply Demand
         "Youngstown":    [10000,0],
         "Pittsburgh":    [15000,0],
         "Cincinatti":    [0,0],
         "Kansas City":   [0,0],
         "Chicago":       [0,0],
         "Albany":        [0,3000],
         "Houston":       [0,7000],
         "Tempe":         [0,4000],
         "Gary":          [0,6000]}

# List of all the arcs
Arcs = [("Youngstown","Albany"),
        ("Youngstown","Cincinatti"),
        ("Youngstown","Kansas City"),
        ("Youngstown","Chicago"),
        ("Pittsburgh","Cincinatti"),
        ("Pittsburgh","Kansas City"),
        ("Pittsburgh","Chicago"),
        ("Pittsburgh","Gary"),
        ("Cincinatti","Albany"),
        ("Cincinatti","Houston"),
        ("Kansas City","Houston"),
        ("Kansas City","Tempe"),
        ("Chicago","Tempe"),
        ("Chicago","Gary")]

arcData = { #      ARC                Cost Min Max
        ("Youngstown","Albany"):      [0.5,0,1000],
        ("Youngstown","Cincinatti"):  [0.35,0,3000],
        ("Youngstown","Kansas City"): [0.45,1000,5000],
        ("Youngstown","Chicago"):     [0.375,0,5000],
        ("Pittsburgh","Cincinatti"):  [0.35,0,2000],
        ("Pittsburgh","Kansas City"): [0.45,2000,3000],
        ("Pittsburgh","Chicago"):     [0.4,0,4000],
        ("Pittsburgh","Gary"):        [0.45,0,2000],
        ("Cincinatti","Albany"):      [0.35,1000,5000],
        ("Cincinatti","Houston"):     [0.55,0,6000],
        ("Kansas City","Houston"):    [0.375,0,4000],
        ("Kansas City","Tempe"):      [0.65,0,4000],
        ("Chicago","Tempe"):          [0.6,0,2000],
        ("Chicago","Gary"):           [0.12,0,4000]}

# Splits the dictionaries to be more understandable
(supply, demand) = splitDict(nodeData)
(costs, mins, maxs) = splitDict(arcData)

# Creates the boundless Variables as Integers
vars = LpVariable.dicts("Route",Arcs,None,None,LpInteger)

# Creates the upper and lower bounds on the variables
for a in Arcs:
    vars[a].bounds(mins[a], maxs[a])

# Creates the 'prob' variable to contain the problem data    
prob = LpProblem("American Steel Problem",LpMinimize)

# Creates the objective function
prob += lpSum([vars[a]* costs[a] for a in Arcs]), "Total Cost of Transport"

# Creates all problem constraints - this ensures the amount going into each node is at least equal to the amount leaving
for n in Nodes:
    prob += (supply[n]+ lpSum([vars[(i,j)] for (i,j) in Arcs if j == n]) >=
             demand[n]+ lpSum([vars[(i,j)] for (i,j) in Arcs if i == n])), "Steel Flow Conservation in Node %s"%n

# The problem data is written to an .lp file
prob.writeLP("AmericanSteelProblem.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
Route_('Chicago',_'Gary') = 4000.0
Route_('Chicago',_'Tempe') = 2000.0
Route_('Cincinatti',_'Albany') = 2000.0
Route_('Cincinatti',_'Houston') = 3000.0
Route_('Kansas_City',_'Houston') = 4000.0
Route_('Kansas_City',_'Tempe') = 2000.0
Route_('Pittsburgh',_'Chicago') = 3000.0
Route_('Pittsburgh',_'Cincinatti') = 2000.0
Route_('Pittsburgh',_'Gary') = 2000.0
Route_('Pittsburgh',_'Kansas_City') = 3000.0
Route_('Youngstown',_'Albany') = 1000.0
Route_('Youngstown',_'Chicago') = 3000.0
Route_('Youngstown',_'Cincinatti') = 3000.0
Route_('Youngstown',_'Kansas_City') = 3000.0
Total Cost of Transportation =  15005.0
